# Do it yourself: 2D filtered back projection

## Simulate demo image and sinogram to be reconstructed

In [ ]:
import array_api_compat.numpy as np
import matplotlib.pyplot as plt
from utils import demo_phantom_and_projector

In [ ]:
# choose number of radial elements, number of views and angular coverage
num_rad = 301
phi_max = np.pi
num_phi = (int(0.5 * num_rad * np.pi * (phi_max / np.pi)) + 1) // 1

In [ ]:
# setup 1D arrays containing the radial and angular coordinates
r = np.linspace(-30, 30, num_rad, dtype=np.float32)
phi = np.linspace(0, phi_max, num_phi, endpoint=False, dtype=np.float32)

In [ ]:
# get: - a demo radon object (an test image where we can calculate the radon transform analytically)
#      - a sinogram (discrete radon transform) of the object
#      - get a projector (line integral calculator) that allows us to reconstruct
radon_object, sino, proj = demo_phantom_and_projector(r, phi)

In [ ]:
# generate a high-resolution discrete ground truth image
x = np.linspace(float(np.min(r)), float(np.max(r)), 1001, dtype=np.float32)
X0hr, X1hr = np.meshgrid(x, x, indexing="ij")
high_res_ground_truth = radon_object.values(X0hr, X1hr)

ext_img = [float(np.min(r)), float(np.max(r)), float(np.min(r)), float(np.max(r))]
ext_sino = [float(np.min(r)), float(np.max(r)), float(np.min(phi)), float(np.max(phi))]

img_kwargs = dict(cmap="Greys", extent=ext_img, origin="lower", vmin = 0)
sino_kwargs = dict(cmap="Greys", aspect=20, extent=ext_sino, origin="lower")

fig, ax = plt.subplots(1, 2, figsize=(8, 4), tight_layout=True)
ax[0].imshow(high_res_ground_truth.T, **img_kwargs)
ax[1].imshow(sino.T, **sino_kwargs)
ax[0].set_title("(discretized) object - ground truth", fontsize="small")
ax[1].set_title("forward projection (sinogram) of object", fontsize="small")

ax[0].set_xlabel(r"$x_0$")
ax[0].set_ylabel(r"$x_1$")
ax[1].set_xlabel(r"$s$")
ax[1].set_ylabel(r"$\phi$")

## Calculate a simple back projection of the sinogram

In [ ]:
# back project the sinogram
# the back projection is the adjoint of the forward projection
back_proj = proj.adjoint(sino)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4), tight_layout=True)
ax.imshow(back_proj.T, **img_kwargs)
ax.set_title("back projection of sinogram", fontsize="small")
ax.set_xlabel(r"$x_0")
ax.set_ylabel(r"$x_1$")

## Setup of the ramp filter

In [ ]:
# setup a discrete ramp filter
n_filter = r.shape[0]
r_shift = np.arange(n_filter, dtype=np.float64) - n_filter // 2
f = np.zeros(n_filter, dtype=np.float64)
f[r_shift != 0] = -1 / (np.pi**2 * r_shift[r_shift != 0] ** 2)
f[(r_shift % 2) == 0] = 0
f[r_shift == 0] = 0.25

In [ ]:
# visualize the dicretized ramp filter
figr, axr = plt.subplots(1, 2, figsize = (9,4.5))
axr[0].plot(np.arange(num_rad) - num_rad//2, f)
axr[1].plot(np.arange(num_rad) - num_rad//2, f, ".-")
axr[1].set_xlim(-6,6)
axr[0].set_title("discretized ramp filter", fontsize = "small")
axr[1].set_title("discretized ramp filter (zoom)", fontsize = "small")
axr[0].grid(ls=':')
axr[1].grid(ls=':')

## Calculate a filtered back projection of the sinogram

In [ ]:
# ramp filter the sinogram in the radial direction - view by view
filtered_sino = np.zeros_like(sino)

for i in range(num_phi):
    filtered_sino[:, i] = np.convolve(sino[:, i], f, mode="same")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4), tight_layout=True)
ax.imshow(filtered_sino.T, **sino_kwargs)
ax.set_title("ramp filtered sinogram", fontsize="small")
ax.set_xlabel(r"$s$")
ax.set_ylabel(r"$\phi$")

In [ ]:
# back project the ramp filtered sinogram
filtered_back_proj = proj.adjoint(filtered_sino)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4), tight_layout=True)
ax.imshow(filtered_back_proj.T, **img_kwargs)
ax.set_title("filtered back projection of sinogram", fontsize="small")
ax.set_xlabel(r"$s$")
ax.set_ylabel(r"$\phi$")

## Visualize all results

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(9, 6), tight_layout=True)
ax[0, 0].imshow(high_res_ground_truth.T, **img_kwargs)
ax[1, 1].imshow(sino.T, **sino_kwargs)
ax[1, 2].imshow(filtered_sino.T, **sino_kwargs)
ax[0, 1].imshow(back_proj.T, **img_kwargs)
ax[0, 2].imshow(filtered_back_proj.T, **img_kwargs)

for axx in ax[0, :].ravel():
    axx.set_xlabel(r"$x_0$")
    axx.set_ylabel(r"$x_1$")
for axx in ax[1, 1:].ravel():
    axx.set_xlabel(r"$s$")
    axx.set_ylabel(r"$\phi$")

ax[1, 0].set_axis_off()

ax[0, 0].set_title("object", fontsize="small")
ax[0, 1].set_title("back projection of sinogram", fontsize="small")
ax[0, 2].set_title("filtered back projection of sinogram", fontsize="small")

ax[1, 1].set_title("radon transform of object - sinogram", fontsize="small")
ax[1, 2].set_title("ramp filtered sinogram", fontsize="small")